In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
print(torch.__version__)
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F

import helper

import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

ModuleNotFoundError: No module named 'torch'

In [2]:
data_dir = '/home/akshay/Projects/flower_classification/flowers/train'
batch_size = 32
tf = transforms.Compose([transforms.Resize((255, 255)),
                         transforms.ToTensor(),
                        ])

train_dataset = datasets.ImageFolder(data_dir, transform = tf)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

data_dir = '/home/akshay/Projects/flower_classification/flowers/test'

test_dataset = datasets.ImageFolder(data_dir, transform = tf)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

In [3]:
m = len(train_loader) * batch_size
n = 255 * 255 * 3
learning_rate = 1e-3
num_epochs = 10

In [9]:
# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size = n, num_classes = 1):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        return out

In [24]:
model = LogisticRegression(input_size = n, num_classes = 1)
model = model.cuda()
# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.BCELoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [25]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(images.shape[0], -1))
        labels = Variable(labels)
        
        images = images.cuda()
        labels = labels.cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        reshaped_labels = labels.resize_(labels.shape[0], 1)
        loss = criterion(outputs, reshaped_labels.float())
        loss.backward()
        optimizer.step()
        

    print ('Epoch: [%d/%d], Loss: %.4f' 
           % (epoch+1, num_epochs, loss.data[0]))

/home/akshay/anaconda3/envs/pyt/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: [1/10], Loss: 1.9009
Epoch: [2/10], Loss: 0.5204
Epoch: [3/10], Loss: 0.3300
Epoch: [4/10], Loss: 0.1850
Epoch: [5/10], Loss: 0.3323
Epoch: [6/10], Loss: 0.1475
Epoch: [7/10], Loss: 0.1902
Epoch: [8/10], Loss: 0.6184
Epoch: [9/10], Loss: 0.2857
Epoch: [10/10], Loss: 0.1737


In [26]:
# Test the Model on training set
correct = 0
total = len(train_loader) * batch_size
for images, labels in train_loader:
    images = Variable(images.view(images.shape[0], -1))
    labels = Variable(labels)
    
    images = images.cuda()
    labels = labels.cuda()
    
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    correct += torch.eq(predicted, labels).sum()
    
print(correct.item() * 100 / total)

# Test the Model
correct = 0
total = len(test_loader) * batch_size
for images, labels in test_loader:
    images = Variable(images.view(images.shape[0], -1))
    labels = Variable(labels)
    
    images = images.cuda()
    labels = labels.cuda()
    
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    correct += torch.eq(predicted, labels).sum()
    
print(correct.item() * 100 / total)

50.07102272727273
61.71875
